# Naive Bayes classifier

- https://drive.google.com/open?id=1zbVKIHMUugqXkKDc4q7CNaWXcwQ7pzFE

請到上面把整個資料下載，共分三個

- chinese_news: 原本的新聞
- chinese_trans: 翻譯成繁體的新聞
- chinese_tests: 我從原本的新聞每個分類擷取出 10 篇當作測試文章

## MS-DOS 執行，以下指令，移除「_DS_Store」檔案
- DEL /S _DS_Store

```console
E:\PyAI\workspace\chinese_news_test>dir "_DS_Store" /s /b
E:\PyAI\workspace\chinese_news_test\交通\_DS_Store
E:\PyAI\workspace\chinese_news_test\政治\_DS_Store
E:\PyAI\workspace\chinese_news_test\教育\_DS_Store
E:\PyAI\workspace\chinese_news_test\環境\_DS_Store
E:\PyAI\workspace\chinese_news_test\經濟\_DS_Store
E:\PyAI\workspace\chinese_news_test\藝術\_DS_Store
E:\PyAI\workspace\chinese_news_test\計算機\_DS_Store
E:\PyAI\workspace\chinese_news_test\軍事\_DS_Store
E:\PyAI\workspace\chinese_news_test\醫藥\_DS_Store
E:\PyAI\workspace\chinese_news_test\體育\_DS_Store

E:\PyAI\workspace\chinese_news_test>del /s _DS_Store
刪除的檔案 - E:\PyAI\workspace\chinese_news_test\交通\_DS_Store
刪除的檔案 - E:\PyAI\workspace\chinese_news_test\政治\_DS_Store
刪除的檔案 - E:\PyAI\workspace\chinese_news_test\教育\_DS_Store
刪除的檔案 - E:\PyAI\workspace\chinese_news_test\環境\_DS_Store
刪除的檔案 - E:\PyAI\workspace\chinese_news_test\經濟\_DS_Store
刪除的檔案 - E:\PyAI\workspace\chinese_news_test\藝術\_DS_Store
刪除的檔案 - E:\PyAI\workspace\chinese_news_test\計算機\_DS_Store
刪除的檔案 - E:\PyAI\workspace\chinese_news_test\軍事\_DS_Store
刪除的檔案 - E:\PyAI\workspace\chinese_news_test\醫藥\_DS_Store
刪除的檔案 - E:\PyAI\workspace\chinese_news_test\體育\_DS_Store

E:\PyAI\workspace\chinese_news_test>dir "_DS_Store" /s /b
找不到檔案
```

In [2]:
import os
import jieba
import pandas as pd
# 為了顯示的漂亮, 我刻意的把印出來的 row 只顯示 15 個和 column 只顯示十個
# 大家練習的時候可以去掉下面兩行
pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', 10)
# scikit-learn 會有些 deprecation warning, 為了顯示漂亮, 我刻意地忽略掉
import warnings
warnings.filterwarnings('ignore')
base_dir = "chinese_news_trans"
test_dir = "chinese_news_test"

# 因為要處理資料夾很多次，所以定義成函式
def process_dirs(base_dir):
    df = pd.DataFrame(columns = ["類別", "內容"])
    # os.walk 會走到檔案才停下來
    for dir_path, dir_names, file_names in os.walk(base_dir):
        for single_file in file_names:
            if not single_file.startswith("."):
                f = open(os.path.join(dir_path, single_file), "r", encoding = "utf-8")
                content = f.read()
                # 讀完黨以後做出第一步處理, 先把換行都去掉
                content = content.replace("\r", "").replace("\n", "")
                split_word = jieba.cut(content)
                # 分詞
                content = " ".join(split_word)
                s = pd.Series([dir_path.split("\\")[-1], content], index = ["類別", "內容"])
                df = df.append(s, ignore_index = True)
    df['類別'] = df['類別'].astype('category')
    return df

In [3]:
df = process_dirs(base_dir)
df

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BIGDAT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.602 seconds.
Prefix dict has been built succesfully.


,類別,內容
0,交通,【 日 期 】 19960104 【 版 號 】 1 【 ...
1,交通,【 日 期 】 19960226 【 版 號 】 5 【 ...
2,交通,大 秦鐵路 萬噸 列車 試運 成功 新華社 北京...
3,交通,遼寧省 檯 安縣 村村 都 通 柏油路 鄉村 公...
4,交通,北京 — 烏蘭 巴托 — 莫斯料 ３ ／ ４ 次...
5,交通,福建 將建 第二 條出 省鐵路 新華社 福州 ５...
6,交通,大秦 二期工程 中 最長 的 平市 東河 特大 ...
...,...,...
2630,體育,參加 首屆 世界 盃 乒乓球 團體賽 的 中國 ...
2631,體育,全國 健美 賽 和 健美操 賽決 出 ６ 項 第...


In [4]:
# 把類別替換成 code, 並且記錄起來
# 走過 categories 同時順便把字典創造起來
saved_map = { cat:df['類別'].cat.categories.get_loc(cat)
             for cat in df['類別'].cat.categories }
saved_map

{'交通': 0,
 '政治': 1,
 '教育': 2,
 '環境': 3,
 '經濟': 4,
 '藝術': 5,
 '計算機': 6,
 '軍事': 7,
 '醫藥': 8,
 '體育': 9}

In [5]:
df['類別'] = df['類別'].cat.codes
df

,類別,內容
0,0,【 日 期 】 19960104 【 版 號 】 1 【 ...
1,0,【 日 期 】 19960226 【 版 號 】 5 【 ...
2,0,大 秦鐵路 萬噸 列車 試運 成功 新華社 北京...
3,0,遼寧省 檯 安縣 村村 都 通 柏油路 鄉村 公...
4,0,北京 — 烏蘭 巴托 — 莫斯料 ３ ／ ４ 次...
5,0,福建 將建 第二 條出 省鐵路 新華社 福州 ５...
6,0,大秦 二期工程 中 最長 的 平市 東河 特大 ...
...,...,...
2630,9,參加 首屆 世界 盃 乒乓球 團體賽 的 中國 ...
2631,9,全國 健美 賽 和 健美操 賽決 出 ６ 項 第...


In [6]:
test_df = process_dirs(test_dir)
test_df

,類別,內容
0,交通,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
1,交通,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
2,交通,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
3,交通,三趟 火車 停開 乘客 可 全額 退票 瀏覽次數 ： 1180 ...
4,交通,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
5,交通,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
6,交通,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
...,...,...
94,體育,中國 青島 － 韓國 大邱 健美 表演 賽 和 對 抗賽 將舉 辦 ...
95,體育,男子 健美 初登亞 運會 中國 猛男 直指 前 三 在 即將...


In [7]:
test_df['類別'] = test_df['類別'].replace(saved_map)
test_df

,類別,內容
0,0,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
1,0,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
2,0,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
3,0,三趟 火車 停開 乘客 可 全額 退票 瀏覽次數 ： 1180 ...
4,0,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
5,0,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
6,0,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
...,...,...
94,9,中國 青島 － 韓國 大邱 健美 表演 賽 和 對 抗賽 將舉 辦 ...
95,9,男子 健美 初登亞 運會 中國 猛男 直指 前 三 在 即將...


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
# 注意一定要使用 fit_transform, 才會幫你轉換成詞向量
bag = vec.fit_transform(df['內容'])
print("總共維度:", len(vec.get_feature_names()))
# 讀者可以把註解拿掉, 就可以看到 transform 後的 matrix
# for entry in bag:
#     print(entry)

總共維度: 96042


In [10]:
# 只要是文字, 我們通常就會選擇 MultinomialNB
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.001).fit(bag, df['類別'])

In [11]:
# 由於我們用剛剛的 vec 訓練他, 所以維度會保持跟剛剛一樣
test_bag = vec.transform(test_df['內容'])
print("維度:", len(vec.get_feature_names()))

維度: 96042


In [12]:
from sklearn.metrics import accuracy_score
predict = clf.predict(test_bag)
print("預測　　:", list(predict))
print("正確標籤:", list(test_df['類別']))
print("Naive-Bayes 正確率: ", accuracy_score(test_df['類別'], predict) * 100, "%")

預測　　: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
Naive-Bayes 正確率:  100.0 %


In [13]:
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(n_neighbors=8)
clf = clf.fit(bag, df['類別'])
predict = clf.predict(test_bag)

print("預測　　:", list(predict))
print("正確標籤:", list(test_df['類別']))
print("kNN 正確率: ", accuracy_score(test_df['類別'], predict) * 100, "%")

預測　　: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
kNN 正確率:  100.0 %
